# Linear model overfitting
* This simple notebook demonstrates how to overfit a linear regression
* It really isn't that hard
* Also, it shows basic problem for linear regresssion in the "big data" world
* It then tries to fix this using Ridge and Lasso

In [1]:
# Load helpers
# Will try to just load what I need on this
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [2]:
# Function to generate linear data experiments
def genLinData(N,M,noise):
    # y = x_1 + x_2 .. x_M + eps
    # X's scaled so the variance of explained part is same order as noise variance (if std(eps) = 1)
    sigNoise = np.sqrt(1./M)
    X = np.random.normal(size=(N,M),loc=0,scale=sigNoise)
    eps = np.random.normal(size=N,loc=0,scale=noise)
    y = np.sum(X,axis=1)+eps
    return X,y

* Model equation:
* $y = \sum_{i=1}^M x_i + \epsilon_i$
* $E(y)=E(x_i) = E(\epsilon)=0$
* $\hat y =  \sum_{i=1}^M x_i$
* $\text{Var}({x_i}) = (1/M)$
* $\text{Var}({\sum x_i}) = 1 = \text{Var} (\hat y)$
* Equals $\epsilon$ or noise variance, if noise = 1
* Variance of y = $\text{Var}(\hat y) + \text{Var}(\epsilon)$
* Variance of y = 1 + 1 = 2
* R-squared = $1-\frac{(N-1)*\text{Var}(\epsilon)}{(N-1)\text{Var}(y)}=( 1 - \frac{1}{2}) = 0.5$

In [3]:
# Set up a system that you might have estimated once in your econometrics class
X, y = genLinData(200,2,1.0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=0)
# Now run regression
# print score, which is R-squared (fit)
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))

0.594622173261824
0.49184435234549695


## Now increase right hand side forecast variables (a lot)

In [4]:
# Set up a system you would be told never to try in your econometrics class
X, y = genLinData(200,50,1.0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=0)
# Now run regression
# print score, which is R-squared (fit)
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))

0.7221767771647211
-0.09568655969851038


## A quick monte-carlo example
* Note: you can do this many times
* This is known as a monte-carlo
* See code below
* There is a big for loop 
* Statistics for each run are stored in numpy vector (scoreVec)

In [5]:
scoreVec = np.zeros(100)
for i in range(100):
    X, y = genLinData(200,50,1.0)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5)
    # Now run regression
    # print score, which is R-squared (fit)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    scoreVec[i] = lr.score(X_test,y_test)
print(np.mean(scoreVec))
print(np.std(scoreVec))
print(np.mean(scoreVec<0))

-0.06395555193541601
0.24810739882055938
0.57


### Increase sample size
* The ultimate solution for overfitting

In [6]:
# Set up a system you would be told never to try in your econometrics class
X, y = genLinData(20000,50,1.0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=0)
# Now run regression
# print score, which is R-squared (fit)
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))

0.5053594300644131
0.5006110188740948


# Overfitting
* This regression is clearly overfitting
* Try many different runs of this
* This is a form of overfitting
* **Note:**  The model is technically the correct model

# Ridge and Lasso ----

# Try Ridge and Lasso
* Control overfitting
* Parameter alpha now added
* Note: In this experiment no coefficients are zero

In [7]:
X, y = genLinData(200,50,1.0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=0)
# Now run regression
# print score, which is R-squared (fit)
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
print(ridge.score(X_train,y_train))
print(ridge.score(X_test,y_test))

0.6644539932566441
0.21595608677509104


In [8]:
X, y = genLinData(200,50,1.0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=0)
# Now run regression
# print score, which is R-squared (fit)
lasso = Lasso(alpha=0.0025)
lasso.fit(X_train, y_train)
print(lasso.score(X_train,y_train))
print(lasso.score(X_test,y_test))

0.7495972396450203
0.21305567850556162


# A more complicated model
* Set some of the parameters to zero
* This may give even more overfitting
* Can also see how well Lasso gets zeros
* Finding irrelevant information

In [9]:
# Function to generate linear data experiments
# Now drop some coefficients to zero (sparse)
def genLinData(N,M,noise):
    # y = x_1 + x_2 .. x_M + eps
    # X's scaled so the variance of explained part is same order as noise variance (if eps = 1)
    sigNoise = np.sqrt(1./M)
    # set up random beta for regression
    beta = np.random.normal(size=(M,1),loc=0.,scale=1.)
    # force smaller beta to zero
    beta[abs(beta)<1.0] = 0.
    betaUsed= np.sum( beta != 0.)
    X = np.random.normal(size=(N,M),loc=0,scale=sigNoise)
    eps = np.random.normal(size=(N,1),loc=0,scale=noise)
    # Modern Python with matrix multiplication
    y = X @ beta + eps
    # Find theoretical best R-squared 
    sse = np.sum(eps**2)
    meany = np.mean(y)
    sse2 = np.sum( (y-meany)**2)
    trsquared = 1. - sse/sse2
    # Old style Python
    # X = np.random.normal(size=(N,M),loc=0,scale=sigNoise)
    # eps = np.random.normal(size=N,loc=0,scale=noise)
    # y = np.sum(X,axis=1)+eps
    return X,y,betaUsed,trsquared

### Ordinary least squares

In [10]:
X, y, nvar, trs = genLinData(200,50,1.0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=0)
# Now run regression
# print score, which is R-squared (fit)
lr = LinearRegression()
lr.fit(X_train, y_train)
print(trs)
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))
print(nvar)
print(lr.coef_)
print(np.mean(np.abs(lr.coef_)))
print(np.sum(lr.coef_!=0))

0.3623177643098451
0.6630147842772923
-0.2614681852302312
11
[[-0.234874    0.03971918  0.32634017 -0.59338448 -3.11108987  0.80041525
   0.18174424  2.72293106  1.61032184  0.26993116  1.97945008 -0.70972809
  -3.48440867  1.04625833  1.91764536  0.99481565  0.35054434 -1.90481734
  -0.57162909  1.99496871  1.52291838  1.22144749  0.94243827 -1.21353232
   1.60256293  0.47984155  0.42780426  0.62576777  1.38493944  0.91529516
   0.52462835  0.71160237 -0.78313485 -0.15311965  1.9133465  -0.41988404
  -0.37285608 -1.18004611  0.06855708 -0.17456601  0.80243753  0.36359459
   0.22190386  0.42904851 -0.82950563 -1.60486922  2.13449515  0.35694226
  -0.72197241 -0.21780494]]
0.983317592177224
50


### Now quick monte-carlo

In [11]:
nmc = 1000
scoreVec = np.zeros(nmc)
scoreVecInSamp = np.zeros(nmc)
trsVec = np.zeros(nmc)
for i in range(nmc):
    X, y, nvar, trs = genLinData(250,50,1.0)
    trsVec[i] = trs
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5)
    # Now run regression
    # print score, which is R-squared (fit)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    scoreVecInSamp[i] = lr.score(X_train,y_train)
    scoreVec[i] = lr.score(X_test,y_test)
print(np.mean(trsVec))
print(np.mean(scoreVecInSamp))
print(np.mean(scoreVec))
print(np.mean(scoreVec<0))


0.43552205285165335
0.662334029841268
0.03047442731798991
0.393


### Ridge regression

In [12]:
X, y, nvar, trs = genLinData(200,50,1.0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=0)
# Now run regression
# print score, which is R-squared (fit)
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
print(trs)
print(ridge.score(X_train,y_train))
print(ridge.score(X_test,y_test))
print(nvar)
print(ridge.coef_)
print(np.mean(np.abs(ridge.coef_)))
print(np.sum(ridge.coef_!=0))

0.5538727291581442
0.6703858582383189
0.26977902601379067
16
[[-0.55840901  0.59240777 -0.44913178 -0.72686675  0.42131789  1.27170035
  -0.51080305 -1.15142713 -0.36628985  0.03354132  0.10587337  0.08297305
  -0.28566255  0.62801971  0.35810673  0.13903004  1.13413042 -1.61236532
  -0.11968789  0.61950813  0.42234329 -0.3411994   0.44469962 -0.60867638
   0.23748332 -0.03132906  0.62059148 -0.32642761  0.41894926  0.43588149
   1.59077311 -0.2058528  -1.6363259  -0.50350675  0.5995654  -0.69209899
   0.93543383 -0.54547788  0.37274492 -1.26079047  0.00425337  0.33390983
   1.3838307   0.01298133 -0.87311486  0.56586691  1.37627098  0.15299158
  -0.11912182 -0.13459007]]
0.5670866901044388
50


### Monte-carlo for Ridge

In [13]:
nmc = 1000
scoreVec = np.zeros(nmc)
scoreVecInSamp = np.zeros(nmc)
trsVec = np.zeros(nmc)
for i in range(nmc):
    X, y, nvar, trs = genLinData(250,50,1.0)
    trsVec[i] = trs
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5)
    # Now run regression
    # print score, which is R-squared (fit)
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train, y_train)
    scoreVecInSamp[i] = ridge.score(X_train,y_train)
    scoreVec[i] = ridge.score(X_test,y_test)
print(np.mean(trsVec))
print(np.mean(scoreVecInSamp))
print(np.mean(scoreVec))
print(np.mean(scoreVec<0))


0.42908497283829583
0.5909055202869715
0.23836805352482052
0.016


### Lasso regression

In [14]:
X, y, nvar, trs = genLinData(200,50,1.0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=0)
# Now run regression
# print score, which is R-squared (fit)
lasso = Lasso(alpha=0.025)
lasso.fit(X_train, y_train)
print(trs)
print(lasso.score(X_train,y_train))
print(lasso.score(X_test,y_test))
print(nvar)
print(lasso.coef_)
print(np.mean(np.abs(lasso.coef_)))
print(np.sum(lasso.coef_!=0))

0.35662063350190165
0.28350409208513905
0.1656704596944899
13
[ 0.         -0.          0.          0.         -0.52623042  0.
  0.          0.         -0.          0.48457794 -0.61271486  0.
  0.         -0.          0.82655622 -1.21371223 -0.          0.
  0.          0.          0.          0.          0.         -0.
  0.         -0.         -0.          0.         -0.75886404  0.
  0.         -0.          0.31681958 -1.10036995 -0.         -0.
  0.          0.          0.          0.          0.          0.
 -0.         -0.          0.          0.         -0.73493616 -0.
  0.04827762 -0.        ]
0.13246118018198014
10


### Monte-carlo for Lasso

In [15]:
nmc = 1000
scoreVec = np.zeros(nmc)
scoreVecInSamp = np.zeros(nmc)
trsVec = np.zeros(nmc)
for i in range(nmc):
    X, y, nvar, trs = genLinData(250,50,1.0)
    trsVec[i] = trs
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5)
    # Now run regression
    # print score, which is R-squared (fit)
    lasso = Lasso(alpha=0.025)
    lasso.fit(X_train, y_train)
    scoreVecInSamp[i] = lasso.score(X_train,y_train)
    scoreVec[i] = lasso.score(X_test,y_test)
print(np.mean(trsVec))
print(np.mean(scoreVecInSamp))
print(np.mean(scoreVec))
print(np.mean(scoreVec<0))

0.4331689925224723
0.30920184876847406
0.15945682588307072
0.023
